### **Install Packages and Libraries**

In [62]:
# Install package
!pip install selenium
!pip install webdriver-manager

# Install library
import datetime as dt
import datetime
import numpy as np
import os
import pandas as pd
from selenium import webdriver

# Install wget
!apt-get install wget

# Download the chromedriver.exe from GitHub
os.system('wget https://github.com/SamapanThongmee/Scrap_SET_Index_Stock_Tickers/blob/main/chromedriver.exe')

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


### **Scrap Tickers**

In [63]:
def get_stock_data(url):
    driver = web_driver()
    driver.get(url)
    data = driver.page_source
    data_df = pd.read_html(data)[2]
    data_df.columns = [c.replace('  (Click to sort Ascending)', '') for c in data_df.columns]
    data_df = data_df.set_index('Symbol')

    return [stock.split()[0] for stock in data_df.index]

categories = {
    'AGRO_AGRI': 'https://www.set.or.th/en/market/index/set/agro/agri',
    'AGRO_FOOD': 'https://www.set.or.th/en/market/index/set/agro/food',
    'CONSUMP_FASION': 'https://www.set.or.th/en/market/index/set/consump/fashion',
    'CONSUMP_HOME': 'https://www.set.or.th/en/market/index/set/consump/home',
    'CONSUMP_PERSON': 'https://www.set.or.th/en/market/index/set/consump/person',
    'FINCIAL_BANK': 'https://www.set.or.th/en/market/index/set/fincial/bank',
    'FINCIAL_FIN': 'https://www.set.or.th/en/market/index/set/fincial/fin',
    'FINCIAL_INSUR': 'https://www.set.or.th/en/market/index/set/fincial/insur',
    'INDUS_AUTO' : 'https://www.set.or.th/en/market/index/set/indus/auto',
    'INDUS_IMM' : 'https://www.set.or.th/en/market/index/set/indus/imm',
    'INDUS_PAPER' : 'https://www.set.or.th/en/market/index/set/indus/paper',
    'INDUS_PETRO' : 'https://www.set.or.th/en/market/index/set/indus/petro',
    'INDUS_PKG' : 'https://www.set.or.th/en/market/index/set/indus/pkg',
    'INDUS_STEEL' : 'https://www.set.or.th/en/market/index/set/indus/steel',
    'PROPCON_CONMAT' : 'https://www.set.or.th/en/market/index/set/propcon/conmat',
    'PROPCON_CONS' : 'https://www.set.or.th/en/market/index/set/propcon/cons',
    'PROPCON_PROP' : 'https://www.set.or.th/en/market/index/set/propcon/prop',
    'PROPCON_PF_REIT' : 'https://www.set.or.th/en/market/index/set/propcon/pf&reit',
    'RESOURC_ENERG' : 'https://www.set.or.th/en/market/index/set/resourc/energ',
    'SERVICE_COMM' : 'https://www.set.or.th/en/market/index/set/service/comm',
    'SERVICE_HELTH' : 'https://www.set.or.th/en/market/index/set/service/helth',
    'SERVICE_MEDIA' : 'https://www.set.or.th/en/market/index/set/service/media',
    'SERVICE_PROF' : 'https://www.set.or.th/en/market/index/set/service/prof',
    'SERVICE_TOURISM' : 'https://www.set.or.th/en/market/index/set/service/tourism',
    'SERVICE_TRANS' : 'https://www.set.or.th/en/market/index/set/service/trans',
    'TECH_ETRON' : 'https://www.set.or.th/en/market/index/set/tech/etron',
    'TECH_ICT' : 'https://www.set.or.th/en/market/index/set/tech/ict'
}

result = {category: get_stock_data(url) for category, url in categories.items()}

dataframes = []

for industry, tickers in result.items():
    data = {'Industry': [industry] * len(tickers), 'Ticker': tickers}
    df = pd.DataFrame(data)
    dataframes.append(df)

final_df = pd.concat(dataframes, ignore_index=True)
final_df[['Industry', 'Sector']] = final_df['Industry'].str.split('_', n=1, expand=True)

industry_mapping = {
    'AGRO': 'Agro & Food Industry',
    'CONSUMP': 'Consumer Products',
    'FINCIAL': 'Financials',
    'INDUS': 'Industrials',
    'PROPCON': 'Property & Construction',
    'RESOURC': 'Resources',
    'SERVICE': 'Services',
    'TECH': 'Technology'
}

sector_mapping = {
    'AGRI': 'Agribusiness',
    'FOOD': 'Food & Beverage',
    'FASION': 'Fashion',
    'HOME': 'Home & Office Products',
    'PERSON': 'Personal Products & Pharmaceuticals',
    'BANK': 'Banking',
    'FIN': 'Finance & Securities',
    'INSUR': 'Insurance',
    'AUTO': 'Automotive',
    'IMM': 'Industrial Materials & Machinery',
    'PAPER': 'Paper & Printing Materials',
    'PETRO': 'Petrochemicals & Chemicals',
    'PKG': 'Packaging',
    'STEEL': 'Steel and Metal Products',
    'CONMAT': 'Construction Materials',
    'CONS': 'Construction Services',
    'PROP': 'Property Development',
    'PF_REIT': 'Property Fund & REITs',
    'ENERG': 'Energy & Utilities',
    'COMM': 'Commerce',
    'HELTH': 'Health Care Services',
    'MEDIA': 'Media & Publishing',
    'PROF': 'Professional Services',
    'TOURISM': 'Tourism & Leisure',
    'TRANS': 'Transportation & Logistics',
    'ETRON': 'Electronic Components',
    'ICT': 'Information & Communication Technology'
}

Ticker = final_df[['Ticker', 'Industry', 'Sector']].reset_index(drop=True)
Ticker['Industry_Name'] = Ticker['Industry'].map(industry_mapping).fillna('N/A')
Ticker['Sector_Name'] = Ticker['Sector'].map(sector_mapping).fillna('N/A')
Ticker

,Ticker,Industry,Sector,Industry_Name,Sector_Name
0,EE,AGRO,AGRI,Agro & Food Industry,Agribusiness
1,GFPT,AGRO,AGRI,Agro & Food Industry,Agribusiness
2,LEE,AGRO,AGRI,Agro & Food Industry,Agribusiness
3,MAX,AGRO,AGRI,Agro & Food Industry,Agribusiness
4,NER,AGRO,AGRI,Agro & Food Industry,Agribusiness
...,...,...,...,...,...
687,SYNEX,TECH,ICT,Technology,Information & Communication Technology
688,THCOM,TECH,ICT,Technology,Information & Communication Technology
689,TKC,TECH,ICT,Technology,Information & Communication Technology
690,TRUE,TECH,ICT,Technology,Information & Communication Technology


### **Export to .csv**

In [64]:
Ticker.to_csv('SET_Index_Ticker.csv', index=False)